In [2]:
!pip install hnswlib

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp310-cp310-linux_x86_64.whl size=2287620 sha256=ca4d86aeae3b1abf754fc03e1b0f04a97db379261fd0f93d5e642907acbc42c1
  Stored in directory: /root/.cache/pip/wheels/af/a9/3e/3e5d59ee41664eb31a4e6de67d1846f86d16d93c45f277c4e7
Successfully built hnswlib


In [11]:
!pip install rank_bm25

In [15]:
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm
import hnswlib
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import json

In [13]:
from rank_bm25 import BM25Okapi
import pandas as pd

In [14]:
def get_top_k_all_data(all_data,scores,k):
    formatted_dict = []
    n=len(all_data)
    for i in range(n):
        temp = {}
        query = all_data[i]['input']
        ndocs = len(all_data[i]['profile'])
        docs = [all_data[i]['profile'][j]['text'] for j in range(ndocs)]
        title = [all_data[i]['profile'][j]['title'] for j in range(ndocs)]
        res = sorted(list(zip(docs,title,scores)),key = lambda x:x[2],reverse=True)
        res = res[:min(k,ndocs)]
        temp['query'] = query
        temp['docs'] = [res[i][0] for i in range(min(k,ndocs))]
        temp['titles']=[res[i][1] for i in range(min(k,ndocs))]
        temp['scores'] = [res[i][2] for i in range(min(k,ndocs))]

        formatted_dict.append(temp)
    return formatted_dict
def write_json(dictionary, file):
    json_object = json.dumps(dictionary, indent=4)
    with open(file, "w") as outfile:
        outfile.write(json_object)
    return
def get_top_k_aip(query,docs,titles,scores,k):
    res = sorted(list(zip(docs,titles,scores)),key = lambda x:x[2],reverse=True)
    aip = ""
    for i in range(min(k,len(scores))):
        ppep = '"'+res[i][1]+'" is the title for "'+ res[i][0] +'", and '
        aip +=ppep
    aip = aip[:-6] + "."+query
    return aip
def get_all_data(filename):
    all_data = []
    with open(filename, 'r') as file:
        json_objects = ijson.items(file, 'item')
        for data in json_objects:
            all_data.append(data)
    return all_data
def fwrite(output,filename):
    f = open(filename,'w+')
    f.write(str(output))
    f.close()
    return

def format_aips(aipfile,outputsfile):
    f = open(aipfile,"r")
    print(aipfile)
    aip = eval(f.read())
    f2 = open(outputsfile)
    y_true = json.load(f2)
    y_true = y_true["golds"]
    y_true = [i['output'] for i in y_true]

    llm_data = pd.DataFrame({
        'text':aip,
        'label':y_true
    })
    op_name = str(aipfile).split('.txt')[0]+"_llm_input.csv"
    llm_data.to_csv(op_name)
    return

In [5]:
def hnsw(all_data,vectorizer):
    aip = []
    all_scores = []
    count=0
    for instance in all_data:
        query = instance['input']
        docs = [instance['profile'][i]['text'] for i in range(len(instance['profile']))]
        titles = [instance['profile'][i]['title'] for i in range(len(instance['profile']))]
        document_vectors = vectorizer.fit_transform(docs).toarray()
        dim = document_vectors.shape[1]
        num_elements = len(docs)
        index = hnswlib.Index(space='cosine', dim=dim)
        index.init_index(max_elements=num_elements, ef_construction=200, M=20)
        index.add_items(document_vectors)
        #decide how much ef
        index.set_ef(200)
        query_vector = vectorizer.transform([query]).toarray()[0]
        max_score_index, distances = index.knn_query(query_vector, k=num_elements)
        max_score_index = max_score_index.tolist()
        distances = distances.tolist()
        res = sorted(list(zip(max_score_index[0],distances[0])))
        scores = [1-res[i][1] for i in range(len(res))]
        all_scores.append(scores)
        title = titles[max_score_index[0][0]]
        op = '"'+title+'" is the title for "'+ docs[max_score_index[0][0]] +'".'+query
        aip.append(op)
        count+=1
        print(count)
    return aip,all_scores

In [7]:
def ensemble(all_data,models):
    n = len(all_data)
    aip = []
    num_models = len(models)
    all_overall = []
    for i in range(n):
        query = all_data[i]['input']
        title = [all_data[i]['profile'][j]['title'] for j in range(len(all_data[i]['profile']))]
        docs  = [all_data[i]['profile'][j]['text'] for j in range(len(all_data[i]['profile']))]
        ndocs = len(docs)
        overall_scores = [0 for i in range(ndocs)]
        for j in range(num_models):
            overall_scores = [overall_scores[k]+models[j][i][k] for k in range(ndocs)]
        overall_scores = [(1/num_models)*overall_scores[k] for k in range(ndocs)]
        aip.append(get_top_k_aip(query,docs,title,overall_scores,5))
        all_overall.append(overall_scores)
        #max_score_index = overall_scores.index(max(overall_scores))
        #op = '"'+txt +'"'+docs[max_score_index]+'" is "'+categories[max_score_index]+'"".'+all_data[i]['input']
        #aip.append(op)
    return aip, all_overall

In [8]:
def bert(all_data, tokenizer, model):
    aip = []
    all_scores=[]
    progress_bar = tqdm(range(101))
    for instance in all_data:
        query = instance['input']
        docs = [instance['profile'][i]['text'] for i in range(len(instance['profile']))]
        title =[instance['profile'][i]['title'] for i in range(len(instance['profile']))]
        document_tokens = tokenizer.batch_encode_plus(docs, padding=True, truncation=True, return_tensors='pt').to('cuda')
        query_tokens = tokenizer.encode_plus(query, padding=True, truncation=True, return_tensors='pt').to('cuda')
        document_embeddings = model(**document_tokens).last_hidden_state.mean(dim=1)  # Average pooling over tokens
        query_embedding = model(**query_tokens).last_hidden_state.mean(dim=1).squeeze()
        similarities = cosine_similarity(query_embedding.reshape(1, -1).detach().cpu().numpy(), document_embeddings.detach().cpu().numpy()).squeeze()
        progress_bar.update(1)
        ranking = sorted(enumerate(similarities), key=lambda x: x[1], reverse=True)
        #print(ranking)
        max_score_index = ranking[0][0]
        op = '"'+title[max_score_index]+'" is the title fxor "'+ docs[max_score_index] +'".'+query
        aip.append(op)
        ranking = sorted(ranking)
        scores = [ranking[i][1] for i in range(len(ranking))]
        #print("SCORES",ranking,scores)
        all_scores.append(scores)
    return aip, all_scores

In [9]:
def bm25_model(all_data):
    aip=[]
    all_scores = []
    #ppep format "Pi[title]" is the title for "Pi[text]"
    for i in range(len(all_data)):
        query = all_data[i]['input']
        docs = [all_data[i]['profile'][j]['text'] for j in range(len(all_data[i]['profile'])) ]
        tokenized_docs = [doc.split() for doc in docs]
        title = [all_data[i]['profile'][j]['title'] for j in range(len(all_data[i]['profile'])) ]
        bm25 = BM25Okapi(tokenized_docs)
        scores = bm25.get_scores(query.split())
        #min max normalization
        min_score = min(scores)
        max_score = max(scores)
        if max_score == min_score:
            scores = [1.0 for i in range(len(scores))]
        else:
            scores = [(score- min_score)/(max_score-min_score)  for score in scores]
        all_scores.append(scores)
        max_score_index = np.argmax(scores)

        #ppep format "Pi[title]" is the title for "Pi[text]"
        op = '"'+title[max_score_index]+'" is the title for "'+ docs[max_score_index] +'".'+query
        aip.append(op)
    return aip,all_scores

In [16]:
def read_output(filename):
    f = open(filename,"r")
    scores= f.read()
    scores = ast.literal_eval(scores)
    #print("LEN",len(scores))
    return scores

In [ ]:

file = 'dev_questions.json'
output_file = 'dev_outputs.json'
all_data = get_all_data(file)

In [ ]:


bm25_s = read_output('bm25/train/bm25_scores_train.txt')
bert_s = read_output('bert/train/bert_scores_train.txt')
hnsw_s = read_output('hnsw/train/hnsw_scores_train.txt')

In [ ]:
models = [[bm25_s,hnsw_s,bert_s],[bm25_s,bert_s],[bert_s,hnsw_s],[bm25_s,hnsw_s]]
#models = [[bm25_s,hnsw_s]]
#names = ['bm25_hnsw_dev']
names = ['bm25_hnsw_bert_dev','bm25_bert_dev','bert_hnsw_dev','bm25_hnsw_dev']
for i in range(len(models)):
#print(len(models[0]))
    aip,overall_scores  = ensemble(all_data,models[i])
    fname = 'ensemble/aip/'+names[i]+'_aip_100.txt'
    fwrite(aip,fname)
    fwrite(overall_scores,'ensemble/scores/'+names[i]+'_scores_100.txt')
    format_aips(fname,'dev_outputs.json')